# **Setup**

Mount drive to get data: 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Download relevant repos:**

In [0]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

In [0]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import itertools
import os
import json
import random
import operator
import torch
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, peak_widths

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog
from detectron2.data.datasets import load_coco_json
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import ColorMode
from detectron2.engine import DefaultTrainer



# **Image Segmentation**

The image segmentation code is based on the tutorial which can be found here: 
https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5 

Load anotations and datafiles, and register as COCO file format:

In [0]:
DatasetCatalog.register("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/",
                        lambda: load_coco_json("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/trainval.json", "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/"))
MetadataCatalog.get("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/trainval.json").set(thing_classes=["femur","patella", "tibia"])

 metadata = MetadataCatalog.get("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/trainval.json").set(thing_classes=["femur","patella", "tibia"])


In [0]:
dataset_dicts  = load_coco_json("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/trainval.json", "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/MOST/")

Setup model, with pretrained weights, and retrain the model:

In [0]:
cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.OUTPUT_DIR = "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/"
cfg.DATASETS.TRAIN = ("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025*10
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)
cfg.MODEL.DENSEPOSE_ON = True
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [0]:
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/", "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set the testing threshold for this model
cfg.DATASETS.TEST = ("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/MOST/", )
predictor = DefaultPredictor(cfg)

Dump model weights as a cfg file format:

In [0]:
import yaml
with open('cfg.yaml', 'w') as f:
  yaml.dump(cfg, f)

Import model retrained model:

In [0]:
from detectron2_repo.projects.DensePose.densepose.config import add_densepose_config

cfg1 = get_cfg()
add_densepose_config(cfg1)
cfg1.merge_from_file("/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/cfg.yaml")
cfg1.OUTPUT_DIR = "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/Validation/"

predictor = DefaultPredictor(cfg1)

# Train the model using early stopping.

First define the loss function, this is done with DICE coefficients:

The DICE coefficient calculation are based on the code which can be found here: 
https://github.com/milesial/Pytorch-UNet/blob/master/dice_loss.py


In [0]:
import torch
from torch.autograd import Function



class DiceCoeff(Function):
    """Dice coeff for individual examples"""

    def forward(self, input, target):
        self.save_for_backward(input, target)
        eps = 0.0001
        self.inter = torch.dot(input.view(-1), target.view(-1))
        self.union = torch.sum(input) + torch.sum(target) + eps

        t = (2 * self.inter.float() + eps) / self.union.float()
        return t

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):

        input, target = self.saved_variables
        grad_input = grad_target = None

        if self.needs_input_grad[0]:
            grad_input = grad_output * 2 * (target * self.union - self.inter) \
                         / (self.union * self.union)
        if self.needs_input_grad[1]:
            grad_target = None

        return grad_input, grad_target


def dice_coeff(input, target):
    """Dice coeff for batches"""
    if input.is_cuda:
        s = torch.FloatTensor(1).cuda().zero_()
    else:
        s = torch.FloatTensor(1).zero_()

    for i, c in enumerate(zip(input, target)):
        s = s + DiceCoeff().forward(c[0], c[1])

    return s / (i + 1)

In [0]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
from PIL import ImageDraw
from torchvision import transforms
import PIL.Image
import PIL
try:
    import Image
except ImportError:
    from PIL import Image

labels_true=["femur","patella", "tibia"]

def polygons_to_mask(img_shape, polygons):
    mask = np.zeros(img_shape, dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    xy = list(map(tuple, polygons))
    PIL.ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask

def merge(list1, list2):   
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))] 
    return merged_list 

def calculate_dice(dataset_dicts, predictor):
  bone_acc=[]
  for d in dataset_dicts[:]:    
    im = cv2.imread(d["file_name"])
    h=d['height']
    w=d['width']
    outputs = predictor(im)
    dap=outputs['instances'].pred_masks.to('cpu')*1
    doop=outputs['instances'].to('cpu')
    label_nr=doop.pred_classes.numpy()
    bop=d['annotations']
    visualizer = Visualizer(im[:, :, ::-1], metadata= metadata, scale=0.5)
    vis = visualizer.draw_instance_predictions(doop)    
    class_names=metadata.get("thing_classes", None)
    classes=doop.pred_classes
    labels = [class_names[i] for i in classes]

    if len(labels) < 2.5: 
      return 0
    label_nr=0
    
    for i in labels:
      predict_bone=dap[label_nr]
      label_nr+=1
      for y in bop:
        if labels_true[y['category_id']]==i:
          
          points=y['segmentation']
          points_np=points[0]
          yval=points_np[1::2]
          xval=points_np[0::2]
          points_m=merge(xval,yval)
          true_bone=polygons_to_mask([h,w],points_m)
          true_bone=true_bone*1
          trans=transforms.ToTensor()
          true_bony=trans(true_bone)
          predict_bony = predict_bone.unsqueeze(0)
          
          predict_bony = predict_bony.unsqueeze(0)
          true_bony = true_bony.unsqueeze(0)
          
          tot = dice_coeff(predict_bony, true_bony)
          dum_dum=true_bone-(predict_bone.numpy()*255)
          sum_dum=sum(sum(dum_dum))
          bone_acc.append(tot)
          if i=='femur':
            femur_acc=tot
          if i=='patella':
            patella_acc=tot
          if i=='tibia':
            tibia_acc=tot
          
          if sum_dum < 0: 
            sum_dum=sum_dum*-1
          resul=(sum(sum(true_bone))-sum_dum)/sum(sum(true_bone))
  return np.mean(bone_acc)

In [0]:
val_error = calculate_dice(validation_dataset, predictor)
iter_nr=100
error_yuppi=[]
stopper = 0

while stopper < 300:
  stopper += 10
  iter_nr+=10
  cfg.SOLVER.MAX_ITER = iter_nr
  cfg.DATASETS.TEST = ("/content/drive/My Drive/LAT_IMGS/Validation/",)   # 
  trainer = DefaultTrainer(cfg)
  trainer.resume_or_load(resume=True)
  trainer.train()
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  

  predictor = DefaultPredictor(cfg)
  tmp = calculate_dice(validation_dataset, predictor)
  error_yuppi.append(tmp)
  if (tmp>val_error):
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    val_error = tmp
    stopper = 0

# **Detection region of interest (ROI)**




In [0]:
cfg = get_cfg()
add_densepose_config(cfg)
cfg.merge_from_file("/content/drive/My Drive/cfg_early_stop.yaml")
cfg.MODEL.WEIGHTS = "/content/drive/My Drive/model_final_early_stop.pth" 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
predictor = DefaultPredictor(cfg)

**Detection of images containing two knees.**

In [0]:
path_dir = "/content/drive/My Drive/LAT_IMGS/CNN model/lat/full/"
path = '/content/drive/My Drive/LAT_IMGS/CNN model/lat/two_knee_crop'
items = os.listdir("/content/drive/My Drive/LAT_IMGS/CNN model/lat/full/")

for d in items:
  im = cv2.imread(os.path.join(path_dir,d))
  outputs = predictor(im)
  dap = outputs['instances'].pred_masks.to('cpu')*1
  dap = dap.permute(1,2,0)


  # Check if more than 3 masks in image
  if len(dap[0,0,:]) > 3:
    tmp = 0

    for i in range(len(dap[0,0,:])):
      tmp += torch.sum(dap[:,:,i], 0)

    tmp = tmp.data.numpy()
    
    peaks = find_peaks(tmp, height = (50,), distance = 500)
    peak_min_idx = np.argmin(peaks[1]['peak_heights'])
    peak_max_idx = np.argmax(peaks[1]['peak_heights'])
    tmp_min_idx = peaks[0][peak_min_idx]

    peak_min_width = np.uint16(np.floor(peak_widths(tmp, peaks= peaks[0])[0][peak_min_idx]))

    if peak_min_idx < peak_max_idx:
      im_copy = im[:,tmp_min_idx + peak_min_width:].copy()

    elif peak_min_idx > peak_max_idx:
      im_copy = im[:,:tmp_min_idx - peak_min_width].copy()

    else:
      print('Error')
      print(d)

    fullpath = os.path.join(path,d)
    cv2.imwrite(fullpath,im_copy)

**ROI Detection**

In [0]:
items = os.listdir("/content/drive/My Drive/LAT_IMGS/CNN model/lat/full")
path_dir = "/content/drive/My Drive/LAT_IMGS/CNN model/lat/full"

p = 0

for d in items:
    p += 1
    if np.mod(p, 100) == 0:
      print(d, p)
    im = cv2.imread(os.path.join(path_dir,d))
    outputs = predictor(im)

    dap=outputs['instances'].pred_masks.to('cpu')*1

    tmp = dap.permute(1,2,0)
    tmp = tmp.data.numpy()
    tmp = np.uint8(tmp*255)
    img = np.zeros(shape = (tmp.shape[0],tmp.shape[1], 1), dtype = np.uint8)

    #sums all the channels to 1 channel
    for i in range(0,dap.shape[2]):
      img[:,:,0] += dap[:,:,i]

    _, thresh = cv2.threshold(img,1,255, cv2.THRESH_BINARY)

    cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    j=1
    # Chekcs the number of contours found
    # If only one or zero contours are present, erode the image untill atleast 
    # 2 contours is found
    if len(cnts) < 2:
      
      kernel = np.ones((5,5),np.uint8)
      while len(cnts) < 2:
        erosion = cv2.erode(img,kernel,iterations = 1)
        img = erosion
        _, thresh = cv2.threshold(img,1,255, cv2.THRESH_BINARY)
        cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[1]
        j += 1

        if j > 10000:
          boolean_factor = False
          break

      dilate = cv2.dilate(img,kernel,iterations = j-1)
      img = dilate

    # Compute the rectangular area of the found contours
    cnts_area = list()
    for i in range(0,len(cnts)):
      cnts_area.append(cv2.contourArea(cnts[i]))

    # Removes the smallest found countour areas untill only the 
    # Untill only the 2 largest areas remain    
    while len(cnts)!=2:
      cnts.pop(np.argmin(cnts_area))
      cnts_area.pop(np.argmin(cnts_area))
    
    extTop, extBot = list(), list()
    
    #Compute the extrema of the found contours
    for i in cnts:
      c = i
      extTop.append(tuple(c[c[:, :, 1].argmin()][0]))
      extBot.append(tuple(c[c[:, :, 1].argmax()][0]))

    tmp1 = (max(extTop, key = operator.itemgetter(1)))
    tmp2 = (min(extBot, key = operator.itemgetter(1)))

    # Compute the center of the ROI
    cXmean = np.uint32(np.floor(np.mean(np.array([tmp1[0],tmp2[0]]))))
    cYmean = np.uint32(np.floor(np.mean(np.array([tmp1[1],tmp2[1]]))))

    ROI = im[np.int16(abs(np.floor(cYmean-im.shape[0]*0.2))):np.int16(abs(np.floor(cYmean+im.shape[0]*0.2))),
              np.int16(abs(np.floor(cXmean-im.shape[1]*0.3))):np.int16(abs(np.floor(cXmean+im.shape[1]*0.3)))].copy()

    path = "/content/drive/My Drive/LAT_IMGS/CNN model/lat/roi/"
    fullpath = os.path.join(path,d)
    cv2.imwrite(fullpath,ROI)

# **Train model on PA and lateral view separately**

The code for training the PA and lateral view separately using Squeezenet is based on code which can be found here: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

Import Imbalanced dataset sampler:

In [0]:
from sampler import ImbalancedDatasetSampler

Import libraries and setup training:

In [0]:
from torchvision import datasets, models, transforms
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
data_dir = "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/CNN model/pa/roi"
model_name = "squeezenet"
num_classes = 5
batch_size = 16
num_epochs = 200
input_size = 224
feature_extract = False

Define one hot encoder:

The one hot encoder are based on the code from the course material. 

In [0]:
def one_hot_encode(idx, batch_size):
    """
    
    One-hot encodes a single word given its index and the size of the vocabulary.
    
    Args:
     `idx`: the index of the given word
     `vocab_size`: the size of the vocabulary
    
    Returns a 1-D numpy array of length `vocab_size`.
    """
    # Initialize the encoded array
    one_hot = np.zeros((len(idx),5))
    
    # Set the appropriate element to one
    for i in range(len(idx)):
      one_hot[i,idx[i]] = 1.0
    return one_hot



Define training loop:

In [0]:
from sklearn.metrics import*
import torch.nn.functional as F
from torch.autograd import Variable

def train_model(model, dataloaders, criterion, optimizer, lr_scheduler, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.type(torch.FloatTensor).to(device)
                labels = torch.from_numpy(one_hot_encode(labels, batch_size)).type(torch.LongTensor).to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
               
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    loss = criterion(outputs, torch.max(labels,1)[1])
                    _, preds = torch.max(outputs, 1)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                labels_data = torch.max(labels.clone().detach(),1)[1]
                preds_data = preds.clone().detach()
                preds_data = preds_data.cpu()
                labels_data = labels_data.cpu()
                running_corrects += f1_score(labels_data, preds_data, average = 'weighted')
                  
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects*batch_size / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} F1-score: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if phase == 'train':
                scheduler.step(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val F1-score: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

Setup and load squeezenet model:

In [0]:
import torch.nn.functional as F

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /root/.cache/torch/checkpoints/squeezenet1_0-a815701f.pth
100%|██████████| 4.79M/4.79M [00:00<00:00, 117MB/s]

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

Create dataloader and dataset:

In [0]:
from albumentations import *
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, num_workers=4,
                                                   sampler = ImbalancedDatasetSampler(image_datasets[x])) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Define parameters to update, and define optimizer and learning rate scheduler:

In [0]:
model_ft = model_ft.to(device)
from torch.optim import lr_scheduler

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode = 'min', patience=30, threshold = 0.001)


Train the model:

In [0]:
criterion= nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,scheduler, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# **Train model and both PA and lateral view**

The code for training the PA and lateral view simultaneously using Squeezenet is based on code which can be found here: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [0]:
from torchvision import datasets, models, transforms
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
data_dir_pa = "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/CNN model/pa/roi"
data_dir_lat = "/content/drive/My Drive/Kvantitativ biologi og sygdomsmodellering/5. Semester/Deep learning/Project/LAT_IMGS/CNN model/lat/roi"
model_name = "squeezenet"
num_classes = 5
batch_size = 16
num_epochs = 200
input_size = 224
feature_extract = False

In [0]:
from sklearn.metrics import*
import torch.nn.functional as F
from torch.autograd import Variable

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    val_loss_history = []
    train_acc_history = []
    train_loss_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for inputs_pa, inputs_lat, labels in dataloaders[phase]:
                inputs_pa = inputs_pa.type(torch.FloatTensor).to(device)
                inputs_lat = inputs_lat.type(torch.FloatTensor).to(device)
                labels = torch.from_numpy(one_hot_encode(labels, batch_size)).type(torch.LongTensor).to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs_pa, inputs_lat)
                    loss = criterion(outputs, torch.max(labels,1)[1])
                    _, preds = torch.max(outputs, 1)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
          
                # statistics
                running_loss += loss.item() * inputs_pa.size(0)
                labels_data = torch.max(labels.clone().detach(),1)[1]
                preds_data = preds.clone().detach()
                labels_data = labels_data.cpu()
                preds_data = preds_data.cpu()
                running_corrects += f1_score(labels_data, preds_data, average = 'weighted')
                  
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects*batch_size / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} F1-score: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)
                scheduler.step(epoch_acc)


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val F1-score: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, val_loss_history, train_acc_history, train_loss_history

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

Setup squeezenet and alter the model a bit to contain fully connected layers:

The setup of the squeezenet with fully connected layers added how to initialize the weights of the fully connected layeres are based on the code which can be found here: https://github.com/KaiyangZhou/deep-person-reid/blob/master/torchreid/models/squeezenet.py 

In [0]:
from __future__ import division, absolute_import
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
from torch.utils import model_zoo as model_zoo

__all__ = ['squeezenet1_0', 'squeezenet1_1', 'squeezenet1_0_fc512']

model_urls = {
    'squeezenet1_0':
    'https://download.pytorch.org/models/squeezenet1_0-a815701f.pth',
    'squeezenet1_1':
    'https://download.pytorch.org/models/squeezenet1_1-f364aa15.pth',
}


class Fire(nn.Module):

    def __init__(
        self, inplanes, squeeze_planes, expand1x1_planes, expand3x3_planes
    ):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(
            squeeze_planes, expand1x1_planes, kernel_size=1
        )
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(
            squeeze_planes, expand3x3_planes, kernel_size=3, padding=1
        )
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat(
            [
                self.expand1x1_activation(self.expand1x1(x)),
                self.expand3x3_activation(self.expand3x3(x))
            ], 1
        )


class SqueezeNet(nn.Module):
    """SqueezeNet.
    Reference:
        Iandola et al. SqueezeNet: AlexNet-level accuracy with 50x fewer parameters
        and< 0.5 MB model size. arXiv:1602.07360.
    Public keys:
        - ``squeezenet1_0``: SqueezeNet (version=1.0).
        - ``squeezenet1_1``: SqueezeNet (version=1.1).
        - ``squeezenet1_0_fc512``: SqueezeNet (version=1.0) + FC.
    """

    def __init__(
        self,
        num_classes,
        version=1.0,
        fc_dims=None,
        dropout_p=None,
        **kwargs
    ):
        super(SqueezeNet, self).__init__()
        #self.loss = loss
        self.feature_dim = 512

        if version == 1.0:
            self.features = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(96, 16, 64, 64),
                Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(512, 64, 256, 256),
            )
       
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = self._construct_fc_layer(fc_dims, 1024, dropout_p)
        self.classifier = nn.Linear(self.feature_dim, num_classes)

        self._init_params()

    def _construct_fc_layer(self, fc_dims, input_dim, dropout_p=None):
        """Constructs fully connected layer
        Args:
            fc_dims (list or tuple): dimensions of fc layers, if None, no fc layers are constructed
            input_dim (int): input dimension
            dropout_p (float): dropout probability, if None, dropout is unused
        """
        if fc_dims is None:
            self.feature_dim = input_dim
            return None

        assert isinstance(
            fc_dims, (list, tuple)
        ), 'fc_dims must be either list or tuple, but got {}'.format(
            type(fc_dims)
        )

        layers = []
        for dim in fc_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.BatchNorm1d(dim))
            layers.append(nn.ReLU(inplace=True))
            if dropout_p is not None:
                layers.append(nn.Dropout(p=dropout_p))
            input_dim = dim

        self.feature_dim = fc_dims[-1]

        return nn.Sequential(*layers)

    def _init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu'
                )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x1,x2):
        f1 = self.features(x1)
        v1 = self.global_avgpool(f1)
        v1 = v1.view(v1.size(0), -1)
        f2 = self.features(x2)
        v2 = self.global_avgpool(f2)
        v2 = v1.view(v2.size(0), -1)
        v3 = torch.cat((v1,v2),dim=1)
        v3 = self.fc(v3)
        y3 = self.classifier(v3)

        return y3
        

def init_pretrained_weights(model, model_url):
    """Initializes model with pretrained weights.
    
    Layers that don't match with pretrained layers in name or size are kept unchanged.
    """
    pretrain_dict = model_zoo.load_url(model_url, map_location=None)
    model_dict = model.state_dict()
    pretrain_dict = {
        k: v
        for k, v in pretrain_dict.items()
        if k in model_dict and model_dict[k].size() == v.size()
    }
    model_dict.update(pretrain_dict)
    model.load_state_dict(model_dict)


def squeezenet1_0(num_classes, pretrained=True, **kwargs):
    model = SqueezeNet(
        num_classes, version=1.0, fc_dims=None, dropout_p=None, **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, model_urls['squeezenet1_0'])
    return model


def squeezenet1_0_fc512(
    num_classes, pretrained=True, **kwargs
):
    model = SqueezeNet(
        num_classes,
        version=1.0,
        fc_dims=[512,256, 128],
        dropout_p=0.5,
        **kwargs
    )
    if pretrained:
        init_pretrained_weights(model, model_urls['squeezenet1_0'])
    return model

In [0]:
model = squeezenet1_0_fc512(5,pretrained = True)

Define the ImageFolder function, and alter it to pull both the PA and lateral view image every time the dataloader calls it. The code below are a copy of the original ImageFolder function just with alterations to the getitem part of the function in order to pull both the PA and lateral view image. 

In [0]:
def default_loader(path):
        from torchvision import get_image_backend
        if get_image_backend() == 'accimage':
            return accimage_loader(path)
        else:
            return pil_loader(path)

In [0]:
from torchvision.datasets.vision import VisionDataset
from PIL import Image
import os
import os.path
import sys
def has_file_allowed_extension(filename, extensions):
    return filename.lower().endswith(extensions)
def is_image_file(filename):   
    return has_file_allowed_extension(filename, IMG_EXTENSIONS)
def make_dataset(dir, class_to_idx, extensions=None, is_valid_file=None):
    images = []
    dir = os.path.expanduser(dir)
    if not ((extensions is None) ^ (is_valid_file is None)):
        raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")
    if extensions is not None:
        def is_valid_file(x):
            return has_file_allowed_extension(x, extensions)
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                if is_valid_file(path):
                    item = (path, class_to_idx[target])
                    images.append(item)
    return images

class DatasetFolder(VisionDataset):
    def __init__(self, root, loader, extensions=None, transform=None,
                 target_transform=None, is_valid_file=None):
        super(DatasetFolder, self).__init__(root, transform=transform,
                                            target_transform=target_transform)
        classes, class_to_idx = self._find_classes(self.root)
        samples = make_dataset(self.root, class_to_idx, extensions, is_valid_file)
        if len(samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                "Supported extensions are: " + ",".join(extensions)))
        self.loader = loader
        self.extensions = extensions
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]
    def _find_classes(self, dir):
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        #original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        path = self.imgs[index][0].split("/roi/")[-1]
        sample_lat = self.loader(os.path.join(data_dir_lat, path))
        if self.transform is not None:
            sample_lat = self.transform(sample_lat)
        # make a new tuple that includes original and the pat 
        return sample, sample_lat, target
    def __len__(self):
        return len(self.samples)
IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)
def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)
class ImageFolder(DatasetFolder):
    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader, is_valid_file=None):
        super(ImageFolder, self).__init__(root, loader, IMG_EXTENSIONS if is_valid_file is None else None,
                                          transform=transform,
                                          target_transform=target_transform,
                                          is_valid_file=is_valid_file)
        self.imgs = self.samples

Import the Imbalanced dataset sampler which are altered a bit to this model:

In [0]:
from sampler_v2 import ImbalancedDatasetSampler

Dataloader:

In [0]:
from albumentations import *
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: ImageFolder(os.path.join(data_dir_pa, x), data_transforms[x]) for x in ['train', 'val']}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, num_workers=4,
                                                   sampler = ImbalancedDatasetSampler(image_datasets[x])) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')

In [0]:
from torch.optim import lr_scheduler
model_ft =model.to(device)
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.01, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode = 'min', patience=30, threshold = 0.001)

In [0]:
criterion= nn.CrossEntropyLoss()
# Train and evaluate
model_ft, val_acc_hist, val_loss_hist, train_acc_hist, train_loss_hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,scheduler, num_epochs=num_epochs, is_inception=(model_name=="inception"))